In [17]:
!pip install flask-ngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [18]:
!curl -s https://ngrok-agent.s3.amazonaws.com/ngrok.asc | sudo tee /etc/apt/trusted.gpg.d/ngrok.asc >/dev/null && echo "deb https://ngrok-agent.s3.amazonaws.com buster main" | sudo tee /etc/apt/sources.list.d/ngrok.list && sudo apt update && sudo apt install ngrok   

deb https://ngrok-agent.s3.amazonaws.com buster main
Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:5 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 https://ngrok-agent.s3.amazonaws.com buster InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:11 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:14 http://ppa.launchpad.net/graphics-dri

In [19]:
!ngrok config add-authtoken 2HK8LVeJFv2vB0Oxs4gpyBYwhDx_DekdQZkN4JT3dFeihNMx

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [20]:
!pip install tensorflow-addons==0.11.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [21]:

from  flask import Flask, jsonify,render_template, request

#import matplotlib.pyplot as plt
#import matplotlib.ticker as ticker
#from sklearn.model_selection import train_test_split
#import nltk.translate.gleu_score as gleu

import unicodedata
import re
import numpy as np
import os
import io
import time
import pickle
import random
import pandas as pd
from tqdm import tqdm

import tensorflow
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import  pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, LSTMCell
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, ReduceLROnPlateau, TensorBoard

import tensorflow_addons as tfa



#Encoder_DEcoder_Attention

In [22]:
class Encoder(tensorflow.keras.Model):

  def __init__(self, vocab_size, output_dim, input_length, lstm_units, batch_size, embedding_matrix=None, em_trainable=False):
    super().__init__()
    self.lstm_units = lstm_units
    self.batch_size = batch_size
    self.en_hidden_state = 0
    self.en_cell_state = 0
    self.en_output = 0

    #define layers
    if embedding_matrix is None:
      self.embedding = Embedding(input_dim=vocab_size, output_dim=output_dim, input_length=input_length, mask_zero=True, name="encoder_embedding")
    else:
      self.embedding = Embedding(input_dim=vocab_size, output_dim=output_dim, input_length=input_length, mask_zero=True,\
                                 weights=[embedding_matrix], trainable=em_trainable, name='encoder_embedding')
      
    self.encoder = LSTM(units=self.lstm_units,return_state=True, return_sequences=True, name='encoder')

  def call(self, input_sequences, states):

    em_output = self.embedding(input_sequences)
    self.en_output, self.en_hidden_state, self.en_cell_state = self.encoder(em_output, initial_state=states)

    return self.en_output, self.en_hidden_state, self.en_cell_state

  def initialize_states(self):

    self.en_hidden_state = tensorflow.zeros(shape=(self.batch_size, self.lstm_units))
    self.en_cell_state = tensorflow.zeros(shape=(self.batch_size, self.lstm_units))

    return [self.en_hidden_state, self.en_cell_state]

class Decoder(tensorflow.keras.Model):
  def __init__(self, vocab_size, output_dim, input_length, dec_units, batch_sz, embedding_matrix=None, em_trainable=False, attention_type='luong'):
    super().__init__()
    self.batch_sz = batch_sz
    self.dec_units = dec_units
    self.attention_type = attention_type
    self.input_length = input_length

    # Define Embedding Layer
    if embedding_matrix is None:
      self.embedding = Embedding(input_dim=vocab_size, output_dim=output_dim, input_length=input_length, name='decoder_embedding')
    else:
      self.embedding = Embedding(input_dim=vocab_size, output_dim=output_dim, input_length=input_length, \
                                 weights=[embedding_matrix], trainable=em_trainable)

    #Final Dense layer on which softmax will be applied
    self.fc = Dense(vocab_size+1)

    # Define the fundamental cell for decoder recurrent structure
    self.decoder_rnn_cell = LSTMCell(self.dec_units)

    # Sampler
    self.sampler = tfa.seq2seq.sampler.TrainingSampler()


    # Create attention mechanism with memory = None
    self.attention_mechanism = self.build_attention_mechanism(self.dec_units, 
                                                              None, self.batch_sz*[self.input_length], self.attention_type)

    # Wrap attention mechanism with the fundamental rnn cell of decoder
    self.rnn_cell = self.build_rnn_cell(batch_sz)

    # Define the decoder with respect to fundamental rnn cell
    self.decoder = tfa.seq2seq.BasicDecoder(self.rnn_cell, sampler=self.sampler, output_layer=self.fc)


  def build_rnn_cell(self, batch_sz):
    rnn_cell = tfa.seq2seq.AttentionWrapper(self.decoder_rnn_cell, 
                                  self.attention_mechanism, attention_layer_size=self.dec_units)
    return rnn_cell

  def build_attention_mechanism(self, dec_units, memory, memory_sequence_length, attention_type='luong'):
    # ------------- #
    # typ: Which sort of attention (Bahdanau, Luong)
    # dec_units: final dimension of attention outputs 
    # memory: encoder hidden states of shape (batch_size, max_length_input, enc_units)
    # memory_sequence_length: 1d array of shape (batch_size) with every element set to max_length_input (for masking purpose)

    if(attention_type=='bahdanau'):
      return tfa.seq2seq.BahdanauAttention(units=dec_units, memory=memory, memory_sequence_length=memory_sequence_length)
    else:
      return tfa.seq2seq.LuongAttention(units=dec_units, memory=memory, memory_sequence_length=memory_sequence_length)

  def build_initial_state(self, batch_sz, encoder_state, Dtype):
    decoder_initial_state = self.rnn_cell.get_initial_state(batch_size=batch_sz, dtype=Dtype)
    decoder_initial_state = decoder_initial_state.clone(cell_state=encoder_state)
    return decoder_initial_state


  def call(self, inputs, initial_state):
    x = self.embedding(inputs)
    outputs, _, _ = self.decoder(x, initial_state=initial_state, sequence_length=self.batch_sz*[self.input_length])
    return outputs

class Encoder_Decoder_Attention(tensorflow.keras.Model):

  def __init__(self, en_vocab_size, en_out_dim, en_input_length, en_lstm, de_vocab_size, de_out_dim, de_input_length, de_lstm, batch_size,\
               attention='loung', en_embeddings=None, en_emb_trainable=False, de_embeddings=None, de_emb_trainable=False):

    super().__init__()
    #obj to encoder
    self.batch_size = batch_size
    self.encoder = Encoder(en_vocab_size, en_out_dim, en_input_length, en_lstm, batch_size, en_embeddings, en_emb_trainable)
    self.decoder = Decoder(de_vocab_size, de_out_dim, de_input_length, de_lstm, batch_size, de_embeddings, de_emb_trainable ,attention)


  def call(self, data):

    input_seq, target_seq = data[0], data[1]

    #get encoder initial states
    enc_states = self.encoder.initialize_states()

    #encod input
    enc_output, en_hidden_state, en_cell_state = self.encoder(input_seq, enc_states)

    #set attention with encoder output
    self.decoder.attention_mechanism.setup_memory(enc_output)

    #get initial ststes for decoder
    de_initial_state = self.decoder.build_initial_state(self.batch_size, [en_hidden_state, en_cell_state], tensorflow.float32)

    #get predictions from decoder
    pred = self.decoder(target_seq, de_initial_state)

    #return the final output
    return pred.rnn_output

#Deploy

In [23]:

#load pre requisities 
MAX_SEQ_LEN = 10

train_encoder_inp = np.load('/content/drive/MyDrive/GrammarCorrection/model/train_encoder_inp.npy')
train_decoder_inp = np.load('/content/drive/MyDrive/GrammarCorrection/model/train_decoder_inp.npy')
train_decoder_out = np.load('/content/drive/MyDrive/GrammarCorrection/model/train_decoder_out.npy')

encoder_embeddings = np.load('/content/drive/MyDrive/GrammarCorrection/model/encoder_embeddings.npy')
decoder_embeddings = np.load('/content/drive/MyDrive/GrammarCorrection/model/decoder_embeddings.npy')


with open('/content/drive/MyDrive/GrammarCorrection/model/tokenizer_encoder_word_level.pickle', 'rb') as fp:
  tokenizer_encoder = pickle.load(fp)

with open('/content/drive/MyDrive/GrammarCorrection/model/tokenizer_decoder_word_level.pickle','rb') as fp:
  tokenizer_decoder = pickle.load(fp)


In [24]:
#optimizer and loss function

optimizer = tensorflow.keras.optimizers.Adam()

#defining custom loss function which will not consider loss for padded zeroes
# code taken from attention assignment
loss_object = tensorflow.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')
def loss_function(real, pred):
  mask = tensorflow.math.logical_not(tensorflow.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tensorflow.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tensorflow.reduce_mean(loss_)


In [25]:
#training parameters
inp_vocab_size = encoder_embeddings.shape[0]
out_vocab_size = decoder_embeddings.shape[0] 

emb_dim = encoder_embeddings.shape[1]

en_inp_len = MAX_SEQ_LEN
de_inp_len = MAX_SEQ_LEN+1

lstm_size=64
batch_size=4

#load model

model = Encoder_Decoder_Attention(inp_vocab_size, emb_dim, en_inp_len, lstm_size,\
                                   out_vocab_size, emb_dim, de_inp_len, lstm_size,\
                                   batch_size,attention='bahdanau',\
                                   en_embeddings=encoder_embeddings, en_emb_trainable=True,
                                   de_embeddings=decoder_embeddings, de_emb_trainable=True)


In [26]:

model.compile(optimizer=optimizer, loss=loss_function)

model.train_on_batch([train_encoder_inp[:batch_size], train_decoder_inp[:batch_size]], train_decoder_out[:batch_size])
model.load_weights('/content/drive/MyDrive/GrammarCorrection/checkpoints/Attention/en_de_bahdanau_trainable_w.h5')

In [27]:
def decontracted(phrase):
    # specific
    phrase = re.sub(r"won\'t", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

def clean(text):
    """
    takes string as input and
    removes characters inside (),{},[] and <>
    removes characters like -+@#^/|*(){}$~`
    we not not removing ,.!-:;"' as these characters are present in english language 
    """
    text = re.sub('<.*>', '', text)
    text = re.sub('\(.*\)', '', text)
    text = re.sub('\[.*\]', '', text)
    text = re.sub('{.*}', '', text)
    text = re.sub("[-+@#^/|*(){}$~<>=_%:;]","",text)
    text = text.replace("\\","")
    text = re.sub("\[","",text)
    text = re.sub("\]","",text)
    text = re.sub("[0-9]","",text)
    text = text.lower()
    text = text.strip()
    return text

def beam_evaluate_sentence(sentences, model, beam_width):

  inputs = tokenizer_encoder.texts_to_sequences(sentences)
  inputs = pad_sequences(inputs, maxlen=MAX_SEQ_LEN, padding='post', truncating='post', value=0)
  inputs = tensorflow.convert_to_tensor(inputs)

  inference_batch_size = inputs.shape[0]
  result=''

  #encoder
  encoder_start_state = [tensorflow.zeros(shape=(inference_batch_size, lstm_size)), tensorflow.zeros(shape=(inference_batch_size, lstm_size))]
  en_out, en_h, en_c = model.layers[0](inputs, encoder_start_state)

  dec_h, dec_c = en_h, en_c

  start_tokens = tensorflow.fill([inference_batch_size], tokenizer_decoder.word_index['<start>'])
  end_token = tokenizer_decoder.word_index['<end>']

  #tile encoder output
  en_out_tiled = tfa.seq2seq.tile_batch(en_out, multiplier=beam_width)
  model.layers[1].attention_mechanism.setup_memory(en_out_tiled)

  #tile encoder hidden state
  hidden_states = tfa.seq2seq.tile_batch([en_h, en_c], multiplier=beam_width)
  decoder_initial_state = model.layers[1].rnn_cell.get_initial_state(batch_size=beam_width*inference_batch_size, dtype=tensorflow.float32)
  decoder_initial_state = decoder_initial_state.clone(cell_state=hidden_states)

  decoder_instance = tfa.seq2seq.BeamSearchDecoder(model.layers[1].rnn_cell, beam_width=beam_width, output_layer=model.layers[1].fc)

  decoder_embedding_matrix = model.layers[1].embedding.variables[0]

  outputs, final_state, seq_lengths = decoder_instance(decoder_embedding_matrix, start_tokens=start_tokens, end_token=end_token,\
                                                       initial_state=decoder_initial_state)
  
  final_outputs = tensorflow.transpose(outputs.predicted_ids, perm=(0,2,1))
  beam_scores = tensorflow.transpose(outputs.beam_search_decoder_output.scores, perm=(0,2,1))

  return final_outputs.numpy(), beam_scores.numpy()

def grammatical_error_correction(sentence, model):
  result, beam_score = beam_evaluate_sentence(sentence, model, beam_width=3)
  output = tokenizer_decoder.sequences_to_texts(result[0])

  return output[0]

#Run

In [ ]:
from flask_ngrok import run_with_ngrok
from flask import Flask, render_template,request
from flask import Flask
import pandas as pd
import joblib
import json
import numpy as np

app = Flask(__name__)

run_with_ngrok(app)

@app.route('/')
def hellword():
  print('Hello word')
  return jsonify({'message':'Hello World!!'})

@app.route('/predict', methods=['post'])
def predict():
  print("Inside predict function")
  sent = request.get_json(force = True)
  print(sent)
  print("Input:  ", sent["1"])
  sent = sent["1"]
  print("---"*30)
  sent = decontracted(sent)
  sent = clean(sent)
  print(sent)
  corrected = grammatical_error_correction([sent], model)

  result = {'CORRECTED':corrected}
  print("output:  ",result)
  return jsonify(result)


app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://91e9-34-125-69-161.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [16/Dec/2022 07:51:34] "GET / HTTP/1.1" 200 -


Hello word
